In [ ]:
from ipyleaflet import *

import junodch_utils_read_img as utils

import matplotlib.pyplot as matPlt
#import gdal

import rasterio
from rasterio import plot as rastPlt

from localtileserver import get_leaflet_tile_layer, TileClient
from geemap import *

In [ ]:
folderName = "img/Test/"
fig, axs = matPlt.subplots(1,4,figsize=(15,15))

axs[0].set_title("Sentinel-2")
with rasterio.open(folderName + "Sentinel-2_1.tif") as r:
    rastPlt.show(r, ax=axs[0])

axs[1].set_xlim(axs[0].get_xlim())
axs[1].set_ylim(axs[0].get_ylim())
axs[2].set_xlim(axs[0].get_xlim())
axs[2].set_ylim(axs[0].get_ylim())
axs[3].set_xlim(axs[0].get_xlim())
axs[3].set_ylim(axs[0].get_ylim())

axs[1].set_title("Landsat-8")
with rasterio.open(folderName + "Landsat-8_1.tif") as r: 
    rastPlt.show(r, ax=axs[1])

axs[2].set_title("VIIRS Nighttime")
with rasterio.open(folderName + "Night VIIRS_1.tif") as r:
    rastPlt.show(r, ax=axs[2])

axs[3].set_title("Human settlement")
with rasterio.open(folderName + "Population GHSL_1.tif") as r:
    rastPlt.show(r, ax=axs[3])


In [ ]:
folderName = "img/Test/"
fig, axs = matPlt.subplots(1,2,figsize=(10,10))

train, test = utils.getTrainingAndTestPerimeter(folderName + "Night VIIRS_1.tif", 200)

with rasterio.open(folderName + "Sentinel-2_1.tif") as l8:
  dataTrain, transformTrain = utils.getImgFromCoord(l8, train, False)
  dataTest, transformTest = utils.getImgFromCoord(l8, test, False)

  rastPlt.show(dataTrain, transform=transformTrain, ax=axs[0])
  rastPlt.show(dataTest, transform=transformTest, ax=axs[1])

# Combine images

In [ ]:
def combineImages(folder, nImg):
    listRaster = []
    for i in range(1, nImg + 1):
        listRaster.append(rasterio.open(folder + '_' + str(i) + '.tif'))
    output_meta = rasterio.open(folder + '_1.tif').meta.copy()

    mosaic, meta = utils.getMosaicFromFiles(listRaster, output_meta)
    
    return mosaic, meta

In [ ]:
mosaic, meta = combineImages('img/Sokoto/Sentinel-2', 64)
rastPlt.show(mosaic)

with rasterio.open('img/Sokoto/Sentinel-2.tif', 'w', **meta) as r:
  r.write(mosaic)

# Overlap

In [ ]:
m = geemap.Map(center=(0, 0), zoom=12)
m.add_control(ScaleControl(position='topright'))
m.add_control(LayersControl())

imgPath = os.path.join(os.getcwd(),'img/Sokoto/Sentinel-2.tif')
m.add_local_tile(imgPath, bands=[4,3,2],layer_name='test')

display(m)

In [ ]:
m = Map(center=(0, 0), zoom=4)

m.add_layer(get_leaflet_tile_layer(TileClient('img/Sokoto/Sentinel-2.tif')))
display(m)